# Blue Brain BioExplorer

![](./images/bioexplorer_banner.png)

### Prerequiries

In [ ]:
# !pip install pyquaternion

### Initialization

In [22]:
from bioexplorer import BioExplorer, Protein, Surfactant, Membrane, Volume, \
                        Cell, Sugars, Vector2, Vector3, Quaternion
from pyquaternion import Quaternion as pyQuaterion
from braynsmediamaker import MovieMaker

be = BioExplorer('localhost:5000')
core = be.core_api()
mm = MovieMaker(core)
status = be.reset()

### Resources

In [23]:
resource_folder = '../tests/test_files/'
pdb_folder = resource_folder + 'pdb/'

glucose_path = pdb_folder + 'glucose.pdb'
lactoferrin_path=pdb_folder + 'immune/1b0l.pdb'
defensin_path = pdb_folder + 'immune/1ijv.pdb'

surfactant_head_source = pdb_folder + 'surfactant/1pw9.pdb'
surfactant_branch_source = pdb_folder + 'surfactant/1k6f.pdb'


glycan_folder = pdb_folder + 'glycans/'
complex_paths = [glycan_folder + 'complex/5.pdb', glycan_folder + 'complex/15.pdb',
                 glycan_folder + 'complex/25.pdb',glycan_folder + 'complex/35.pdb']
high_mannose_paths = [glycan_folder + 'high-mannose/1.pdb', 
                      glycan_folder + 'high-mannose/2.pdb',
                      glycan_folder + 'high-mannose/3.pdb',
                      glycan_folder + 'high-mannose/4.pdb']
hybrid_paths = [glycan_folder + 'hybrid/20.pdb']
o_glycan_paths = [glycan_folder + 'o-glycan/1.pdb']

### Configuration

In [24]:
# Scene
scene_size = 250.0

# Proteins
protein_radius_multiplier = 1.0
protein_representation = be.REPRESENTATION_ATOMS
protein_load_hydrogen = False

# Virus
add_glycans = True

## Camera and rendering settings

In [25]:
status = be.core_api().set_camera(
    orientation=[0.0, 0.0, 0.0, 1.0],
    position=[4.883, 44.255, 431.911],
    target=[4.883, 44.255, 31.311]
)

In [26]:
be.core_api().set_renderer(
    background_color=[96 / 255, 125 / 255, 139 / 255],
    current='bio_explorer',
    samples_per_pixel=1, subsampling=4, max_accum_frames=64)
params = be.core_api().BioExplorerRendererParams()
params.gi_samples = 1
params.gi_weight = 0.3
params.gi_distance = 500
params.shadows = 1.0
params.soft_shadows = 0.3
params.fog_start = 1500
params.fog_thickness = 1500
params.max_bounces = 1
status = be.core_api().set_renderer_params(params)

## Movie

In [32]:
key_frames = [
    {
        'apertureRadius': 0.0,
        'direction': [-1.0, 0.0, 0.0],
        'focusDistance': 0.0,
        'origin': [461.35815561123326, 33.59808446214642, 0.9491999386753934],
        'up': [0.0, 1.0, 0.0]
    },
    {
        'apertureRadius': 0.0,
        'direction': [-1.0, 0.0, 0.0],
        'focusDistance': 0.0,
        'origin': [145.56207765382672, -66.33695188520096, 66.94663230017692],
        'up': [0.0, 1.0, 0.0]
    },
    {
        'apertureRadius': 0.0,
        'direction': [0.019348546834322612, -0.20204906829112498, -0.9791842562756452],
        'focusDistance': 0.0,
        'origin': [95.56390849140138, -57.95622236745069, 110.43564693595732],
        'up': [0.00199646527930243, 0.979373449795766, -0.202048657410854]
    },
    {
        'apertureRadius': 0.0,
        'direction': [0.0, 0.0, -1.0],
        'focusDistance': 0.0,
        'origin': [37.48804470210077, -5.321877795630035, 273.42426806725484],
        'up': [0.0, 1.0, 0.0]
    }
]
mm.build_camera_path(key_frames, 100, 10)

### Cell, Coronavirus and Surfactant-D

In [33]:
def add_cell(random_position_seed, random_orientation_seed):
    name='Cell'

    '''ACE2 receptor definition'''
    ace2_receptor = Protein(
        sources=[pdb_folder + '6m18.pdb'],
        occurences=1,
        position=Vector3(0.0, 6.0, 0.0))

    '''Membrane definition'''
    membrane_size = scene_size
    membrane_height = scene_size / 10.0
    membrane = Membrane(
        sources=[pdb_folder + 'membrane/popc.pdb'],
        occurences=40000)

    '''Cell definition'''
    cell = Cell(
        name=name,
        size=membrane_size,
        shape=be.ASSEMBLY_SHAPE_SINUSOIDAL,
        membrane=membrane, receptor=ace2_receptor,
        random_position_seed=random_position_seed,
        random_position_strength=scene_size / 500.0,
        random_orientation_seed=random_orientation_seed,
        random_orientation_strength=0.5,
        extra_parameters=[membrane_height]
    )

    '''Add cell to scene'''
    status = be.add_cell(
        cell=cell, representation=protein_representation,
        atom_radius_multiplier=protein_radius_multiplier,
        position=Vector3(0.0, -80.0, 0.0),
        random_seed=1
    )

    '''Color scheme'''
    be.set_protein_color_scheme(
        name, name + '_' + be.NAME_RECEPTOR, 
        be.COLOR_SCHEME_CHAINS, 'OrRd_r', 7)
    
    '''Glycans'''
    be.add_multiple_glycans(
        representation=protein_representation, assembly_name=name, 
        glycan_type=be.NAME_GLYCAN_COMPLEX,
        protein_name=be.NAME_RECEPTOR, paths=complex_paths, 
        indices=[53, 90, 103, 322, 432, 690])

    be.add_multiple_glycans(
        representation=protein_representation, assembly_name=name,
        glycan_type=be.NAME_GLYCAN_HYBRID,
        protein_name=be.NAME_RECEPTOR, paths=hybrid_paths, 
        indices=[546])

    indices = [[164, Quaternion(0.707, 0.0, 0.707, 0.0)],
               [739, Quaternion(0.707, 0.0, 0.707, 0.0)]]
    for index in indices:
        o_glycan_name = name + '_' + be.NAME_GLYCAN_O_GLYCAN + '_' + str(index[0])
        o_glycan = Sugars(
            assembly_name=name, name=o_glycan_name, 
            source=o_glycan_paths[0],
            protein_name=name + '_' + be.NAME_RECEPTOR, 
            representation=protein_representation,
            chain_ids=[2, 4], site_indices=[index[0]], 
            orientation=index[1])
        be.add_sugars(o_glycan)
    return status

In [34]:
def add_surfactant_d(name, position, orientation, random_seed):
    surfactant_d = Surfactant(
        name=name, surfactant_protein=be.SURFACTANT_PROTEIN_D, 
        head_source=surfactant_head_source,
        branch_source=surfactant_branch_source)
    be.add_surfactant(
        surfactant=surfactant_d, 
        representation=protein_representation,
        atom_radius_multiplier=protein_radius_multiplier,
        position=position, orientation=orientation, 
        random_seed=random_seed)

In [35]:
def add_glucose(random_seed, nb_glucoses=14400):
    volume_position = Vector3(0.0, scene_size / 2.0 - 80.0, 0.0)
    glucose = Protein(
        sources=[glucose_path], load_non_polymer_chemicals=True, 
        occurences=nb_glucoses)
    volume = Volume(
        name=be.NAME_GLUCOSE, size=scene_size, 
        protein=glucose,
        random_position_seed=random_seed,
        random_position_stength=scene_size / 500.0,
        random_orientation_seed=random_seed,
        random_orientation_stength=0.3
    )
    return be.add_volume(
        volume=volume, 
        representation=be.REPRESENTATION_ATOMS_AND_STICKS,
        atom_radius_multiplier=protein_radius_multiplier,
        position=volume_position)

In [36]:
'''Coronavirus'''
q_c_start = pyQuaterion(0.707, 0.707, 0.0, 0.0)
q_c_end = pyQuaterion(1.0, 0.0, 0.0, 0.0)
q_c_nb_frames = 100

'''Surfactant-D'''
q_s_start = pyQuaterion(0.0, 0.0, 0.707,0.707)
q_s_end = pyQuaterion(1.0, 0.0, 0.0, 0.0)
q_s_nb_frames = 300

In [39]:
from IPython.display import clear_output

name='Coronavirus'
nb_frames = mm.get_nb_frames()

be.set_general_settings(model_visibility_on_creation=False, off_folder='/tmp')
core.set_application_parameters(image_stream_fps=0)

for frame in range(23, nb_frames, 1):
    clear_output()
    print('Frame %i out of %i' % (frame, nb_frames))
    
    '''Camera'''
    if True:
        mm.set_current_frame(frame)
    
    if True:
        '''Cell'''
        add_cell(frame, frame + 1)
    
    if True:
        '''Coronavirus'''
        if frame <= q_c_nb_frames:
            orientation = pyQuaterion.slerp(q_c_start, q_c_end, (frame + 1) / q_c_nb_frames)
            be.add_coronavirus(
                name=name, resource_folder=resource_folder, 
                representation=protein_representation, 
                atom_radius_multiplier=protein_radius_multiplier,
                add_glycans=add_glycans,
                position=Vector3(75.0, 100.0 - frame, 10.0 + frame / 2.0),
                orientation=Quaternion(
                    orientation[0],orientation[1],orientation[2],orientation[3])
            )

    if True:
        '''Surfactant-D'''
        orientation = pyQuaterion.slerp(q_s_start, q_s_end, (frame + 1) / q_s_nb_frames)
        add_surfactant_d(
            name='Surfactant-D 1', random_seed=1,
            position=Vector3(
                2.0,
                18.0 + (nb_frames - frame) / 10.0, 
                102.0 + (nb_frames - frame / 2.0) / 10.0),
            orientation = Quaternion(
                    orientation[0],orientation[1],orientation[2],orientation[3])
        )

    if True:
        '''Glucose'''
        add_glucose(frame)
    
    '''Materials'''
    be.apply_default_color_scheme(shading_mode=be.SHADING_MODE_BASIC, user_parameter=1)
    be.set_models_visibility(True)

    if True:
        '''Snapshot'''
        mm.create_snapshot(
            size=[960, 540], samples_per_pixel=32,
            path='/tmp/%05d.png' % frame)

core.set_application_parameters(image_stream_fps=20)

Frame 300 out of 301


IntProgress(value=0, max=34)

IntProgress(value=0, description='In progress...')

True

In [ ]:
add_glucose(7)
be.set_models_visibility(True)

In [ ]:
mm.set_camera(
    direction=[0.0, 0.0, -1.0],
    origin=[3.3480441570281982, -72.81361389160156, 274.4114074707031],
    up=[0.0, 1.0, 0.0]
)